In [21]:
import pandas as pd
import numpy as np


#Reads the tables from the Excel file, the excel is in html format I don't why people do this things...
tables= pd.read_html("REM.xls", encoding="utf-8")

#Options to show the whole dataframe
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

#Gets the first table from the html 
df = tables[0]

#Detects the separators ("OTRAS" rows in the html). This separates the df in blocks where every block represents the data of a location
c0 = df.iloc[:, 0].astype(str).str.strip().str.upper()
c1 = df.iloc[:, 1].astype(str).str.strip().str.upper()
sep = (c0 == "OTRAS") & (c1 == "OTRAS")

#Gets the index of the separators to append it to the blocks. It appends it to the end of the block
idx_sep = np.where(sep.to_numpy())[0]
cuts = (idx_sep + 1).tolist()

# Gets the blocks of data
blocks = [chunk.reset_index(drop=True) for chunk in np.split(df, cuts) if len(chunk) > 0]





/home/benjapc/.local/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [29]:
#Show the first block of data
blocks[0]

MOTIVO DE INGRESO  \
                                                                                      MOTIVO DE INGRESO   
0                                                                                    Acumulado Servicio   
1                                                                                  INGRESOS AL PROGRAMA   
2                                                FACTORES DE RIESGO Y CONDICIONANTES DE LA SALUD MENTAL   
3                                                                                             VIOLENCIA   
4                                                                                             VIOLENCIA   
5                                                                                          ABUSO SEXUAL   
6                                                                                              SUICIDIO   
7                                                                                              SUICIDIO   
8                                                      PERSONAS CON DIAGNÓSTICOS DE TRASTORNOS MENTALES   
9                                                                      TRASTORNOS DEL HUMOR (AFECTIVOS)   
10                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
11                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
12                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
13                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
14                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
15                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
16                                                                     TRASTORNOS DEL HUMOR (AFECTIVOS)   
17                   TRASTORNOS MENTALES Y DEL COMPORTAMIENTO DEBIDO A CONSUMO SUSTANCIAS PSICOTRÓPICAS   
18                   TRASTORNOS MENTALES Y DEL COMPORTAMIENTO DEBIDO A CONSUMO SUSTANCIAS PSICOTRÓPICAS   
19                   TRASTORNOS MENTALES Y DEL COMPORTAMIENTO DEBIDO A CONSUMO SUSTANCIAS PSICOTRÓPICAS   
20  TRASTORNOS DEL COMPORTAMIENTO Y DE LAS EMOCIONES DE COMIENZO HABITUAL EN LA INFANCIA Y ADOLESCENCIA   
21  TRASTORNOS DEL COMPORTAMIENTO Y DE LAS EMOCIONES DE COMIENZO HABITUAL EN LA INFANCIA Y ADOLESCENCIA   
22  TRASTORNOS DEL COMPORTAMIENTO Y DE LAS EMOCIONES DE COMIENZO HABITUAL EN LA INFANCIA Y ADOLESCENCIA   
23  TRASTORNOS DEL COMPORTAMIENTO Y DE LAS EMOCIONES DE COMIENZO HABITUAL EN LA INFANCIA Y ADOLESCENCIA   
24                                                                               TRASTORNOS DE ANSIEDAD   
25                                                                               TRASTORNOS DE ANSIEDAD   
26                                                                               TRASTORNOS DE ANSIEDAD   
27                                                                               TRASTORNOS DE ANSIEDAD   
28                                                                               TRASTORNOS DE ANSIEDAD   
29                                                                               TRASTORNOS DE ANSIEDAD   
30                                                                       DEMENCIAS (INCLUYE ALZHEIMER))   
31                                                                       DEMENCIAS (INCLUYE ALZHEIMER))   
32                                                                       DEMENCIAS (INCLUYE ALZHEIMER))   
33                                                                                        ESQUIZOFRENIA   
34                                                                TRASTORNOS DE LA CONDUCTA ALIMENTARIA   
35                                                                                       RETRASO MENTAL   
36                  

In [23]:
locations = []
# Get the location names from the blocks, the last block is garbage that we will ignore.We skip the "Comuna" blocks beacuse they
# have the same information (they group certain locations).
for i in range(1, len(blocks) - 1):
    if "Comuna" in blocks[i].iloc[0, 0]:
        continue
    location = blocks[i].iloc[0, 0].replace("Establecimiento: ", "")
    locations.append(location)

locations

['Centro de Salud Familiar Placeres',
 'Centro de Salud Familiar Esperanza',
 'Centro de Salud Familiar Barón',
 'Centro de Salud Familiar Reina Isabel II',
 'Centro de Salud Familiar Placilla',
 'Centro de Salud Familiar Jean y Marie Thierry',
 'Centro de Salud Familiar Las Cañas',
 'Centro de Salud Familiar Marcelo Mena',
 'Centro de Salud Familiar Plaza Justicia',
 'Centro de Salud Familiar Cordillera',
 'Centro de Salud Familiar Quebrada Verde',
 'Centro de Salud Familiar Puertas Negras',
 'Centro de Salud Familiar San José de Rodelillo',
 'Centro de Salud Familiar Padre Damián Molokai',
 'Posta de Salud Rural Laguna Verde',
 'Centro Comunitario de Salud Familiar Porvenir Bajo',
 'Hospital San José (Casablanca)',
 'Posta de Salud Rural Quintay',
 'Posta de Salud Rural Las Dichas',
 'Posta de Salud Rural Los Maitenes',
 'Posta de Salud Rural Lagunillas',
 'Centro de Salud Familiar Juan Fernández',
 'Hospital Claudio Vicuña ( San Antonio)',
 'Centro de Salud Familiar San Antonio',
 '

In [24]:
from dotenv import load_dotenv
import os
import googlemaps

#load environment variables
load_dotenv()

#Gets the Google API KEY
google_key = os.getenv("GOOGLE_MAPS_API_KEY")

#Creates googlemaps Client
gmaps = googlemaps.Client(key=google_key)
#Create a dictionary to hold the coordinates, the key being the location name and the value is the coords of the location in a tuple (lat,lng)
locations_coords={}

for location in locations:
    #Get the geocoding result, which contains the location's coordinates.
    geocode_result = gmaps.geocode(location + ", Chile")
    if geocode_result:
        loc = geocode_result[0]["geometry"]["location"]
        lat,lng = loc["lat"], loc["lng"]
    else:
        lat,lng = None, None

    locations_coords[location] = (lat,lng)

locations_coords["Centro de Salud Familiar Placeres"]


(-33.045526, -71.583478)

In [27]:
# Iterate over rows and then columns, and accumulate results into a new DataFrame
def process_block(df_block):
    """
    Build a tidy (long-form) DataFrame from a single location block.

    Assumptions:
    - df_block.iloc[0, 0] contains the location name prefixed by 'Establecimiento: '.
    - Data starts on row index 3 (0-based). Rows 0..2 are headers/labels.
    - Metric columns are positions 5..38 (inclusive) in the DataFrame.
    - The columns form a MultiIndex like (age_group, gender).
    - For each data row, the first two cells (row[0], row[1]) describe the 'cause'.

    Returns:
    - pandas.DataFrame with columns:
      ['gender', 'age_group', 'cause', 'location', 'latitude', 'longitude'].
    """
    # Derive location metadata (name and precomputed coordinates)
    location_name = df_block.iloc[0, 0].replace("Establecimiento: ", "")
    coords = locations_coords.get(location_name, (None, None))

    rows = []
    # Iterate data rows only (skip first 3 label rows)
    for i, (idx, row) in enumerate(df_block.iloc[3:].iterrows(), start=3):
        if i == 8: continue  # Skip the this row beacuse it is a subtotal row
        # For each row, iterate metric columns only (5..38) pairing column names with their values
        for j, (col, val) in enumerate(zip(df_block.columns[5:39], row.iloc[5:39]), start=5):
            # Skip NaN and empty strings
            if pd.isna(val) or (isinstance(val, str) and not val.strip()):
                continue

            # Here, 'col' is a tuple (age_group, gender) from the MultiIndex columns
            # row[0] and row[1] hold descriptive fields that we keep as the 'cause' tuple
            for k in range(int(val)):
                rows.append({
                    "gender" : col[1],
                    "age_group": col[0],
                    "cause": (row[0], row[1]),
                    "location": location_name,
                    "latitude": coords[0],
                    "longitude": coords[1],
                })

    # Assemble and return the normalized DataFrame
    return pd.DataFrame(rows)

In [ ]:
#Gets DataFrame for each block and puts them in a list
df_list = [process_block(block) for block in blocks[1:-1] if "Comuna" not in block.iloc[0, 0]]
# Concatenate all DataFrames into a single DataFrame
df_formated = pd.concat(df_list, ignore_index=True)

df_formated.info()

/tmp/ipykernel_8394/3257665057.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "cause": (row[0], row[1]),


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582 entries, 0 to 6581
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gender     6582 non-null   object 
 1   age_group  6582 non-null   object 
 2   cause      6582 non-null   object 
 3   location   6582 non-null   object 
 4   latitude   6582 non-null   float64
 5   longitude  6582 non-null   float64
dtypes: float64(2), object(4)
memory usage: 308.7+ KB
